In [110]:
#First I need to convert Pdf into image file for that I will use the pdf2image pakage, convert_from_path function

from pdf2image import convert_from_path

pages = convert_from_path(r'docs/patient_details/pd_2.pdf',poppler_path=r'C:\poppler-22.04.0\Library\bin')
pages[0].show()

#Image is too light so can not extract the details properly, so we apply some image process tools
#and then try to convert image to text
for that follow the step

1) we convert image into gray image, b'cuse if any one scan the color image
2) we resize the image for propper convertion image to text
3) After image resize we apply adaptive threshold for better computer vision
4) return the processed image


In [132]:


import numpy as np
import cv2
from PIL import Image

def process_image(img):
    gray = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2GRAY)
  #  img = cv2.imread("dark_image.jpg",flags=cv2.IMREAD_GRAYSCALE)
    resized = cv2.resize(gray,None,fx=1.5,fy=1.5,interpolation = cv2.INTER_LINEAR)
   # processed_image = cv2.threshold(gray,150,255,cv2.THRESH_BINARY)
    processed_image = cv2.adaptiveThreshold(resized,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,51,11)
    return processed_image
    

In [133]:
processed_img = process_image(pages[0])
processed_img

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [134]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text=pytesseract.image_to_string(processed_img,lang='eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date

Jerry Lucas May 2 1998

(279) 920-8204 . Weight:

4218 Wheeler Ridge Dr 57

Buffalo, New York, 14201 Height:

United States gnt.
170

In Case of Emergency |
eee

Joe Lucas . 4218 Wheeler Ridge Dr
Buffalo, New York, 14201
Home phone United States
Work phone

General Medical History

Chicken Pox (Varicelia): Measles: .

IMMUNE NOT IMMUNE

Have you had the Hepatitis B vaccination?

‘Yes:

| List any Medical Problems (asthma, seizures, headaches):
N/A

7?
v




In [150]:
doc_text ='''17/12/2020

Patient Medical Record

Patient Information Birth Date

Kathy Crawford May 6 1972

(737) 988-0851 Weight’

9264 Ash Dr 95

New York City, 10005 '

United States Height:
190

In Case of Emergency
ee J
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
nn ee
Chicken Pox (Varicella): Measies:
IMMUNE

IMMUNE
Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine'''

In [141]:
doc_text_jerry='''Patient Medical Record

Patient Information Birth Date

Jerry Lucas May 2 1998

(279) 920-8204 Weight:

4218 Wheeler Ridge Dr 57

Buffalo, New York, 14201 Height:

United States gnt
170

In Case of Emergency

- eee

Joe Lucas . 4218 Wheeler Ridge Dr
Buffalo, New York, 14201
Home phone United States
Work phone

General Medical History

Chicken Pox (Varicelia): Measles: ..

IMMUNE NOT IMMUNE

Have you had the Hepatitis B vaccination?

Yes

| List any Medical Problems (asthma, seizures, headaches):
N/A

7?
v

17/12/2020'''

Now we use the regular expression and fetch the required filed from text
name,phone,hepitise , anymedical problem

## Extract Name

In [116]:
import re

pattern = 'Patient Information(.*?)\(\d{3}\)'
match = re.findall(pattern,doc_text,flags=re.DOTALL)

name = match[0].replace('Birth Date','')
name.strip()

print(name)

match2 = re.findall(pattern,doc_text_jerry,flags=re.DOTALL)
print(match2[0].strip())

 

Kathy Crawford May 6 1972


Birth Date

Jerry Lucas May 2 1998


In [117]:
print(name)
pattern = '((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
matches = re.findall(pattern,name)
date= matches[0][0]
print(date)

 

Kathy Crawford May 6 1972


May 6 1972


In [118]:
name = name.replace(date," ")
name = name.strip()
print(name)

Kathy Crawford


In [119]:
def remove_noise_from_name(name):
    pattern = '((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    matches = re.findall(pattern,name)
    date= matches[0][0]
    name =name.replace(date," ").strip()
    
    return name

In [120]:
text = "Jerry Lucas May 2 1998"
name = remove_noise_from_name(text)
print(name)

Jerry Lucas


## Extract Phone number

In [124]:
pattern = 'Patient Information.*?(\(\d{3}\) \d{3}-\d{4})'
match = re.findall(pattern,doc_text,re.DOTALL)
print(match[0])
match2 = re.findall(pattern,doc_text_jerry,re.DOTALL)
print(match2[0])

(737) 988-0851
(279) 920-8204


##Extract Vaccination

In [144]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'
match = re.findall(pattern,doc_text,flags=re.DOTALL)
print(match[0].strip())

match2 = re.findall(pattern,doc_text_jerry,flags=re.DOTALL)
match2

No


['Yes']

## Extract medical Problem

In [152]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\)\:(.*)'
match = re.findall(pattern,doc_text,flags=re.DOTALL)
match[0].strip()


'Migraine'

In [65]:
print(f'Hi this is  ',match[0] ,'=')

Hi this is   

Migraine =


In [106]:
text = 'Yes\n\n    |'
t = text.strip('\n,|')
print(t)

Yes

    
